# NAS for CNN

## Structural elements

- Branches
- Connections
- Layers
    - Input
        - Input shape (first input is standard, all others are calculated)
        - Batch size 
    - 2D convolutional
        - Filters
        - Kernel size
        - Padding
        - Activation
    - Pooling
        - Max
            - Size
            - Strides
            - Padding
        - Average
            - Size
            - Strides
            - Padding
    - Dropout
        - Rate
    - Batch normalisation
    - Concatenate
        - Axis
    - Flatten
    - Dense
        - Units (1, 2)
        - Activation (sigmoid, softmax)
    - Global pooling
        - Max
        - Average

## Genetic elements

- GA
    - Type
    - Number of generations
- Population
    - Size
    - Encoding type
    - Individual size
        - Variable or fixed
- Fitness
- Selection
- Reproduction
- Mutation

In [1]:
import numpy as np
import tensorflow as tf
from scipy.spatial import distance
from tensorflow.keras import backend
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPool2D, AveragePooling2D, Conv2D, Input, SpatialDropout2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from functions import *
from IPython.display import clear_output
import yaml

In [ ]:
individual = [
    # Input layer
    'batch_size_index', 
    
    # Convolutional layer
    'kernel_index', 'filter_index', 'padding_index', 'activation_index',

    # Pooling layer
    'type_index', 'size_index', 'padding_index',

    # Dropout layer
    'type_index', 'rate_index',

    # Output layer
    'type_index',

    # Add layer
    ## Layer/Branch 1
    'indicator', 'branch_indicator', 'type', 'from', 'to',

    ## Layer/Branch 2
    'indicator', 'branch_indicator', 'type', 'from', 'to',

    ## ... variable length array

    ## Layer/Branch n
    'indicator', 'branch_indicator', 'type', 'from', 'to'


    ]

In [5]:
class LoadData:

    def __init__(self, path):
        self.path = path

        print_function('Retrieving yaml parameters')

        with open(self.path, "r") as stream:
            self.load_yaml = yaml.load(stream, yaml.SafeLoader)

        self.gen_params = self.load_yaml['general']

        self.seed_value = self.gen_params['seed_value']

        self.random_state = np.random.RandomState(self.seed_value)

        self.data_params = self.load_yaml['data']

        self.sample_size = self.data_params['data_sample_size']

        self.test_set_size = self.data_params['test_set_size']

        self.test_val_split = self.data_params['test_val_split']
        

    def load_data(self):

        print_function('Loading data')

        X = np.load(self.data_params['X_path'])
        
        Y = np.load(self.data_params['Y_path'])

        print('Total sample size', X.shape[0])

        if self.sample_size != None:
            g_sample = int(np.floor(self.sample_size/2))
            n_sample = self.sample_size - g_sample

            g_i = np.random.randint(0, int(X.shape[0]/2) - 1, g_sample)
            n_i = np.random.randint(int(X.shape[0]/2), X.shape[0] - 1, n_sample)

            X = np.concatenate([X[g_i], X[n_i]])
            Y = np.concatenate([Y[g_i], Y[n_i]])
            self.X_train, X_val, self.Y_train, Y_val = train_test_split(X, Y, test_size=self.test_set_size, random_state=self.seed_value, shuffle = True)
            self.X_test, self.X_val, self.Y_test, self.Y_val = train_test_split(X_val, Y_val, test_size=self.test_val_split, random_state=self.seed_value, shuffle = True)

        else:
            self.X_train, X_val, self.Y_train, Y_val = train_test_split(X, Y, test_size=self.test_set_size, random_state=self.seed_value, shuffle = True)
            self.X_test, self.X_val, self.Y_test, self.Y_val = train_test_split(X_val, Y_val, test_size=self.test_val_split, random_state=self.seed_value, shuffle = True)

        print_function('Data successfully loaded')

        print('Training sample size   = ', self.X_train.shape[0])
        print('Validation sample size = ', self.X_val.shape[0])
        print('Testing sample size    = ', self.X_test.shape[0])

        print_function('If you would like to reload the data set argument "load_data_bool = True".')

In [6]:
class GeneticBlockCNN(LoadData):

    def __init__(self, path, load_data_bool = False):

        self.load_data_bool = load_data_bool
        
        super().__init__(path)

        try:
            self.X_train
            if self.load_data_bool:
                super().load_data()
        except:
            super().load_data()

        len_lists = []
        len_values = [] # stores the maximum lengths of bits required to define the length of the lists in the parameters dict
        # max_ind = []
        # bit_max_int = []
        # bit_max = ''

        self.cnn_params = self.load_yaml['cnn_layers']

        self.ga_params = self.load_yaml['genetic_algo']

        self.crossover_params = self.ga_params['crossover']

        print_function('Retrieving state space parameters')

        for layer in self.cnn_params.keys():

            for value_list in list(self.cnn_params[layer].values()):

                # max_ind.append(len(value_list) - 1)

                # bit_max = bit_max + bin(len(value_list) - 1)[2:]

                len_values.append(len(bin(len(value_list) - 1)[2:])) 

                len_lists.append(len(value_list))
            

        self.len_lists = np.array(len_lists)
        
        self.gene_length = np.max(len_values)
        
        self.individual_length = len(self.len_lists) * self.gene_length

        # len_values = np.array(len_values)

        self.population_size = self.ga_params['population_size']

        self.hamming_sample_probability = self.ga_params['hamming_sample_probability']

        print_function('\n', parameter_lengths = self.len_lists, gene_length = self.gene_length, chromosome_length = self.individual_length,
                            population_size = self.population_size)
        
        

    def generate_population(self):
        """
        Generates population given the number of individuals in a population and the required binary length.
        """

        bit_length = int(self.individual_length/len(self.len_lists))

        population = np.zeros(shape=(self.population_size, self.individual_length), dtype=int)
        
        for i in range(self.population_size):
            
            bits = ''
            for l in self.len_lists:

                choice = np.random.choice(np.arange(0, l))

                bit_choice = bin(choice)[2:]

                bits += ('0' * (bit_length - len(bit_choice))) + bit_choice

            population[i] = np.array([int(x) for x in bits])
        
        self.population = population


    
    def decoder(self, chromosome):

        """
        Converts the binary bitstring vector to integer phenotype.
        """

        c = chromosome.astype(int)

        start_ind = np.arange(0, self.individual_length, self.gene_length)

        self.phenotype = np.array([int(str(''.join(map(str, c[x:x + self.gene_length]))), 2) for x in start_ind])

        return self.phenotype



    def validity_check(self):

        """
        Checks whether a phenotype is valid by comparing to the maximum length of values determined using the set of parameters. Returns True or False.
        """

        self.valid_phenotype =  np.all(self.phenotype <= self.len_lists - 1)



    def hamming_distance(self):

        """
        Calculates the mean hamming distance for a sample of the input population.
        """

        n_select = int(np.ceil(self.population_size*self.hamming_sample_probability))

        mean_hamming = 0

        for individual in self.random_state.choice(np.arange(0, self.population_size, n_select), n_select):

            mean_hamming += np.mean([distance.hamming(self.population[individual], self.population[x]) for x in range(n)])

        self.mean_hamming =  mean_hamming/n_select


    
    def get_cnn_parameters(self):

        """
        Fetches the CNN input parameters based on the phenotype.
        """

        input_params = []

        layers = list(self.cnn_params.keys())

        param_count = 0

        for i in range(len(layers)):

            for param_type in self.cnn_params[layers[i]].keys():
                
                input_params.append(self.cnn_params[layers[i]][param_type][self.phenotype[param_count]])
                
                param_count += 1
        
        self.cnn_input_params = input_params

        return self.cnn_input_params



    def train_model(self):

        self.epochs = self.gen_params['epochs']
        
        self.model = Sequential()

        self.model.add(Conv2D(input_shape = self.X_train.shape[1:], 
                            kernel_size = self.cnn_input_params[1], 
                            filters = self.cnn_input_params[2], 
                            padding = self.cnn_input_params[3],
                            activation = self.cnn_input_params[4]))

        if self.cnn_input_params[8] == 'dropout':
            self.model.add(Dropout(rate = self.cnn_input_params[9]))

        else:
            self.model.add(SpatialDropout2D(rate = self.cnn_input_params[9]))

        if self.cnn_input_params[5] == 'max':
            self.model.add(MaxPool2D(pool_size=self.cnn_input_params[6], padding=self.cnn_input_params[7]))

        else:
            self.model.add(AveragePooling2D(pool_size=self.cnn_input_params[6], padding=self.cnn_input_params[7]))

        if self.cnn_input_params[10] == 'dense':
            self.model.add(Flatten())
            self.model.add(Dense(100, activation = self.cnn_input_params[4]))
            self.model.add(Dropout(rate = self.cnn_input_params[9]))


        else:
            self.model.add(GlobalAveragePooling2D())
            self.model.add(Flatten())
        
        self.model.add(Dense(2, activation = 'softmax'))
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', f1_m, precision_m, recall_m])

        self.history = self.model.fit(self.X_train, self.Y_train, validation_data=(self.X_val, self.Y_val), 
                batch_size=self.cnn_input_params[0],
                epochs=self.epochs,
                verbose=self.gen_params['verbose'], 
                class_weight={0:1.,1:1.})



    def check_parent_similarity(self, parent_population):
        return  np.any(np.sum(np.abs(np.diff(parent_population, axis = 0)), axis = 1) == 0)



    def selection(self):

        self.individual_length

        n_selected = 0

        self.selection_params = self.ga_params['selection']

        self.converge_limit = self.ga_params['converge_limit']

        k = self.selection_params['k']

        self.n_parents = self.crossover_params['n_parents']

        selection_probability = self.selection_params['probability']

        self.selected_population_size = int(np.ceil((self.population_size*self.selection_params['population_size_ratio'])/self.n_parents)*self.n_parents) # ensures the selected population size is divisible by the number of parents used in crossover

        similar_parents = True

        convergence_iteration = 0

        while n_selected < self.selected_population_size or convergence_iteration <= self.converge_limit:

            sub_population = self.population[self.random_state.choice(np.arange(0, self.population_size, 1), k, replace = False)]

            scores = []

            iteration = 1

            for individual in sub_population:

                phenotype = self.decoder(individual)

                params = self.get_cnn_parameters()

                self.train_model()

                scores.append(np.mean(self.history.history['val_f1_m'])) # fitness(np.mean(np.diff(self.history.history['val_f1_m'])))

                iteration += 1

            
            rank_index = np.argsort(scores)

            rank_scores = np.array(scores)[rank_index]

            ranked = sub_population[rank_index]

            p_array = np.concatenate(([selection_probability], selection_probability*((1 - selection_probability)**rank_scores)))

            p_array = np.concatenate((p_array[p_array < 1], np.array([1 - np.sum(p_array[p_array < 1])])))

            p_array = np.concatenate((p_array[0:len(ranked) - 1], [np.sum(p_array[len(ranked) - 1:])]))[rank_index]

            selected_index = np.random.choice(rank_index, p = p_array)

            selected = ranked[selected_index]

            fitness_val = rank_scores[selected_index]

            if n_selected == 0:
                
                selected_population = selected

                selected_fitness = np.array([fitness_val])

                n_selected = 1

            else:

                selected_population = np.vstack((selected_population, selected))

                similar_parents = self.check_parent_similarity(selected_population)

                if similar_parents:
                    convergence_iteration += 1
                    continue

                else:
                    selected_fitness = np.concatenate((selected_fitness, np.array([fitness_val])))

                    n_selected = selected_population.shape[0]


            print_function('', Selected = n_selected, of = '', total =  self.selected_population_size)

        self.selected_population = selected_population
        self.selected_fitness = selected_fitness



    def crossover(self):

        crossover_probability = self.crossover_params['probability']

        size_tuple = (self.n_parents, self.individual_length)

        n_offspring = 0

        self.offspring_population_size = self.selected_population_size

        while n_offspring < self.offspring_population_size:

            similar_parents = True

            while similar_parents:

                parent_index = self.random_state.choice(np.arange(0, self.selected_population_size, 1), self.n_parents, replace = False)

                parents = self.selected_population[parent_index]

                similar_parents = self.check_parent_similarity(parents)

            r = self.random_state.random()

            if crossover_probability < 1.0 and crossover_probability > 0.0 and r > crossover_probability:

                continue

            else:

                offspring = np.zeros(size_tuple)

                for i in range(self.n_parents):

                    valid_individual = False

                    while not valid_individual:

                        random_matrix = self.random_state.random(size_tuple)

                        normalised_random_matrix = random_matrix/random_matrix.sum(axis = 0)

                        max_index_j = np.argmax(normalised_random_matrix, axis = 0)

                        offspring_v = parents[max_index_j, range(self.individual_length)]

                        phentotype = self.decoder(offspring_v)

                        self.validity_check()

                        valid_individual = self.valid_phenotype
                        
                    offspring[i] = offspring_v

            print_function('Generated', Offspring = n_offspring + 1, of = '', total = self.offspring_population_size)

            if n_offspring == 0:

                offspring_population = offspring

                n_offspring = 1

            else:

                offspring_population = np.vstack((offspring_population, offspring))

            n_offspring = offspring_population.shape[0]

            

        self.offspring_population = offspring_population
        # self.offspring_population_size = offspring_population_size
    


    def mutation(self):

        self.mutation_params = self.ga_params['mutation']

        mutation_probability = self.mutation_params['probability']

        for i in range(self.offspring_population_size):

            valid_individual = False

            while not valid_individual:

                r = self.random_state.random(self.individual_length)

                mutate_index = (r < mutation_probability)

                offspring_v = np.copy(self.offspring_population[i])

                offspring_v[mutate_index] = np.logical_not(offspring_v[mutate_index])

                phenotype = self.decoder(offspring_v)

                self.validity_check()

                valid_individual = self.valid_phenotype

            self.offspring_population[i] = offspring_v



    def genetic_algorithm(self):

        self.n_generations = self.ga_params['n_generations']

        print_function('Genetic algorithm parameters \n', GA_params = self.ga_params)

        print_function('Generating initital population')

        self.generate_population()

        self.generation_history = {}

        for generation in range(1, self.n_generations + 1):

            print_function('', Generation = generation, of = '', total = self.n_generations)

            print_function('Selection')
            
            self.selection()

            print_function('Crossover')

            self.crossover()

            print_function('Mutation')

            self.mutation()

            # offspring_fitness = []

            # for individual in self.offspring_population:

            #     phenotype = self.decoder(individual)

            #     cnn_params = self.get_cnn_parameters()

            #     self.train_model()

            #     offspring_fitness.append(np.mean(np.diff(self.history.history['val_f1_m'])))

            # total_pool = np.vstack((self.selected_population, self.offspring_population))

            # total_fitness = np.concatenate((self.selected_fitness, np.array(offspring_fitness)))

            # new_population_index = np.argsort(total_fitness)[::-1][:self.population_size]

            # population = total_pool[new_population_index]

            self.population = np.vstack((self.selected_population, self.offspring_population))

            self.generation_history[str(generation)] = {}

            self.generation_history[str(generation)]['selected_population'] = self.selected_population

            self.generation_history[str(generation)]['selected_fitness'] = self.selected_fitness

            self.generation_history[str(generation)]['offspring'] = self.offspring_population

            if generation < self.n_generations:
                clear_output(wait=True)

        print_function('Run completed')

        return self.generation_history
  

    # def fitness(fitness_input, population = None, model_params = None, age = None, beta = 10):

    #     diversity = 1

    #     n_params = 1

    #     if population != None:
    #         diversity = hamming_distance(population)

    #     if model_params != None:
    #         n_params = 1 # need to determine the maximum number of possible parameters so that we can scale

    #     return 1/(1 + np.exp(-beta*fitness_input)) # * 1/diversity * n_params
    

In [7]:
my_cnn = GeneticBlockCNN('parameters.yaml')

Retrieving yaml parameters...
__________________________________________________
Loading data...
__________________________________________________
Total sample size 73160
Data successfully loaded...
__________________________________________________
Training sample size   =  600
Validation sample size =  200
Testing sample size    =  200
If you would like to reload the data set argument "load_data_bool = True"....
__________________________________________________
Retrieving state space parameters...
__________________________________________________

...
-- parameter_lengths .......... [3 4 4 2 4 2 4 2 2 9 2]
-- gene_length ...................................... 4
-- chromosome_length ............................... 44
-- population_size ................................. 20
__________________________________________________


In [9]:
gen_hist = my_cnn.genetic_algorithm()

...
-- Generation ....................................... 4
-- of ................................................ 
-- total ........................................... 10
__________________________________________________
Selection...
__________________________________________________
Epoch 1/10


2022-12-04 16:08:39.674923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-04 16:08:41.858513: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


75/75 - 3s - loss: 0.6740 - acc: 0.5767 - f1_m: 0.5767 - precision_m: 0.5767 - recall_m: 0.5767 - val_loss: 0.6366 - val_acc: 0.5950 - val_f1_m: 0.5950 - val_precision_m: 0.5950 - val_recall_m: 0.5950 - 3s/epoch - 42ms/step
Epoch 2/10
75/75 - 2s - loss: 0.5947 - acc: 0.7050 - f1_m: 0.7050 - precision_m: 0.7050 - recall_m: 0.7050 - val_loss: 0.6111 - val_acc: 0.7150 - val_f1_m: 0.7150 - val_precision_m: 0.7150 - val_recall_m: 0.7150 - 2s/epoch - 27ms/step
Epoch 3/10
75/75 - 2s - loss: 0.5140 - acc: 0.8000 - f1_m: 0.8000 - precision_m: 0.8000 - recall_m: 0.8000 - val_loss: 0.5879 - val_acc: 0.7550 - val_f1_m: 0.7550 - val_precision_m: 0.7550 - val_recall_m: 0.7550 - 2s/epoch - 25ms/step
Epoch 4/10
75/75 - 2s - loss: 0.4947 - acc: 0.8050 - f1_m: 0.8050 - precision_m: 0.8050 - recall_m: 0.8050 - val_loss: 0.6111 - val_acc: 0.7100 - val_f1_m: 0.7100 - val_precision_m: 0.7100 - val_recall_m: 0.7100 - 2s/epoch - 26ms/step
Epoch 5/10
75/75 - 2s - loss: 0.4317 - acc: 0.8250 - f1_m: 0.8250 - pre

2022-12-04 16:08:57.657438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-04 16:08:59.065224: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


75/75 - 2s - loss: 0.6935 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6944 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 2s/epoch - 26ms/step
Epoch 2/10
75/75 - 1s - loss: 0.6936 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6946 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 1s/epoch - 15ms/step
Epoch 3/10
75/75 - 1s - loss: 0.6935 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6947 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 1s/epoch - 15ms/step
Epoch 4/10
75/75 - 1s - loss: 0.6935 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6944 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 1s/epoch - 15ms/step
Epoch 5/10
75/75 - 1s - loss: 0.6939 - acc: 0.5133 - f1_m: 0.5133 - pre

...

2022-12-04 16:20:42.651602: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-04 16:20:44.360270: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


75/75 - 2s - loss: 0.6936 - acc: 0.5183 - f1_m: 0.5183 - precision_m: 0.5183 - recall_m: 0.5183 - val_loss: 0.6939 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 2s/epoch - 32ms/step
Epoch 2/10
75/75 - 1s - loss: 0.6935 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6943 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 1s/epoch - 16ms/step
Epoch 3/10
75/75 - 1s - loss: 0.6934 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6942 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 1s/epoch - 16ms/step
Epoch 4/10
75/75 - 1s - loss: 0.6935 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6946 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 1s/epoch - 16ms/step
Epoch 5/10
75/75 - 1s - loss: 0.6934 - acc: 0.5133 - f1_m: 0.5133 - pre

2022-12-04 16:20:56.219932: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-04 16:20:59.145764: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 - 4s - loss: 0.6994 - acc: 0.5517 - f1_m: 0.5493 - precision_m: 0.5493 - recall_m: 0.5493 - val_loss: 0.6856 - val_acc: 0.5400 - val_f1_m: 0.5385 - val_precision_m: 0.5385 - val_recall_m: 0.5385 - 4s/epoch - 114ms/step
Epoch 2/10
38/38 - 2s - loss: 0.6735 - acc: 0.6317 - f1_m: 0.6332 - precision_m: 0.6332 - recall_m: 0.6332 - val_loss: 0.6520 - val_acc: 0.7400 - val_f1_m: 0.7500 - val_precision_m: 0.7500 - val_recall_m: 0.7500 - 2s/epoch - 64ms/step
Epoch 3/10
38/38 - 2s - loss: 0.6238 - acc: 0.6317 - f1_m: 0.6266 - precision_m: 0.6266 - recall_m: 0.6266 - val_loss: 0.6524 - val_acc: 0.6450 - val_f1_m: 0.6490 - val_precision_m: 0.6490 - val_recall_m: 0.6490 - 2s/epoch - 62ms/step
Epoch 4/10


KeyboardInterrupt: 

In [11]:
gen_hist = my_cnn.generation_history

In [14]:
for gen in gen_hist.keys():
    index = np.argmax(gen_hist[gen]['selected_fitness'])
    fitness = np.max(gen_hist[gen]['selected_fitness'])
    phenotype = my_cnn.decoder(gen_hist[gen]['selected_population'][index])
    print(phenotype, ' ', fitness)

[1 2 3 0 1 1 0 1 0 4 1]   0.7096153378486634
[0 2 2 0 3 1 1 0 0 1 1]   0.7204999625682831
[1 3 3 1 1 1 0 1 0 5 1]   0.7048076510429382


In [14]:
def fitness(fitness_input, population = None, model_params = None, age = None, beta = 10):

    diversity = 1

    n_params = 1

    if population != None:
        diversity = hamming_distance(population)

    if model_params != None:
        n_params = 1 # need to determine the maximum number of possible parameters so that we can scale

    return 1/(1 + np.exp(-beta*fitness_input)) # * 1/diversity * n_params


In [21]:
my_cnn.model.summary()

Model: "sequential_105"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_105 (Conv2D)         (None, 128, 188, 16)      800       
                                                                 
 dropout_125 (Dropout)       (None, 128, 188, 16)      0         
                                                                 
 average_pooling2d_50 (Avera  (None, 64, 94, 16)       0         
 gePooling2D)                                                    
                                                                 
 flatten_105 (Flatten)       (None, 96256)             0         
                                                                 
 dense_143 (Dense)           (None, 100)               9625700   
                                                                 
 dropout_126 (Dropout)       (None, 100)               0         
                                                    